In [0]:
MODEL_NAME = 'od-{epoch:02d}-{val_loss:.4f}-647aug.h5'
EPOCH_NUM=10
BASE_LR=1e-4
MAX_LR=6e-4
STEP_SIZE=3000.
MODE='triangular2'
IMG_SIZE = (224,224)

In [0]:
import os
from glob import glob
os.chdir('/content/')
!rm -rf Satellite-Segmentation
!git clone https://github.com/JasonChu1313/Satellite-Segmentation.git
import os 
os.chdir('/content/Satellite-Segmentation')
!mkdir /root/.kaggle/
!mv kaggle.json /root/.kaggle/

!kaggle competitions download -c comp-540-spring-2019
!mkdir data
!mv train.zip ./data/
!mv val.zip ./data/
os.chdir("./data")
!unzip val.zip
!unzip train.zip

Cloning into 'Satellite-Segmentation'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 431 (delta 89), reused 141 (delta 79), pack-reused 278
Receiving objects: 100% (431/431), 815.60 KiB | 2.09 MiB/s, done.
Resolving deltas: 100% (291/291), done.
 40% 9.00M/22.5M [00:00<00:00, 14.2MB/s]
100% 22.5M/22.5M [00:00<00:00, 35.4MB/s]
 98% 462M/470M [00:07<00:00, 69.3MB/s]
100% 470M/470M [00:07<00:00, 67.5MB/s]
 82% 73.0M/88.6M [00:01<00:00, 32.3MB/s]
100% 88.6M/88.6M [00:01<00:00, 59.2MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
Archive:  val.zip
   creating: val/
  inflating: val/68707_sat.jpg       
  inflating: val/68711_sat.jpg       
  inflating: val/68720_sat.jpg       
  inflating: val/68729_sat.jpg       
  inflating: val/68731_sat.jpg       
  inflating: val/68733_sat.jpg       
  inflating: val/68738_sat.jpg      

In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# https://drive.google.com/open?id=1Zro2_M7FJvMNgpMFqcuvcb-HGcP86c8v
model = drive.CreateFile({'id':'1Zro2_M7FJvMNgpMFqcuvcb-HGcP86c8v'})
model.GetContentFile('cp-id4-37-sb-coef.csv')

    100% |████████████████████████████████| 993kB 20.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
!mkdir 647outlier

mkdir: cannot create directory ‘647outlier’: File exists


In [0]:
import os
import csv
import os
from glob import glob
from skimage.io import imread, imsave
import numpy as np

path_to_train = './train/'
outlier_path = './647outlier/'
dicescore_csv = "./cp-id4-37-sb-coef.csv"

def move_outlier():
    csv_file = open(dicescore_csv, 'r')
    all_lines = csv_file.readlines()
    for line in all_lines[1:647]:
        list = line.split(',')
        img_id = list[0]
        img = imread(path_to_train+img_id+'_sat.jpg')
        imsave(outlier_path+img_id+'_out.jpg', img)
        imsave(outlier_path + img_id + 'a_out.jpg', img[:,::-1])
        imsave(outlier_path + img_id + 'b_out.jpg', img[::-1,:])
        os.remove(path_to_train + img_id + '_sat.jpg')

def choose_normal():
    normal_paths = glob(path_to_train + "/*_sat.jpg")
    normal_imgs = np.random.choice(normal_paths, 3*647)
    for path in normal_imgs:
        img = imread(path)
        img_basename = os.path.basename(path)
        img_id = os.path.splitext(img_basename)[0][:-len('_sat')]
        imsave(outlier_path+img_id+'_sat.jpg', img)

move_outlier()
choose_normal()

FileNotFoundError: ignored

In [0]:
import os
from glob import glob
from skimage.io import imread, imsave
from skimage.transform import resize
import numpy as np
from skimage.color import rgb2gray

path_to_train = '../data/train'
def get_img_id(img_path):
  img_basename = os.path.basename(img_path)
  img_id = os.path.splitext(img_basename)[0][:-len('_sat')]
  return img_id


def image_gen1(img_paths, img_size=(512, 512)):
  #Iterate over all image paths
  for img_path in img_paths:
    img_id = get_img_id(img_path)
    mask_path = os.path.join(path_to_train,img_id+'_msk.png')
    img = imread(img_path)
    mask = imread(mask_path)
    
#     img = resize(img, img_size, preserve_range = True)
#     mask = resize(mask, img_size, mode='constant', preserve_range = True)
    
    yield img, mask, img_id

    
def aug_outlier(outlier_path, normal_path):
    '''
    This function iterate the training set and select outlier and save to outlier path
    and non-outlier save to normal path
    Then randomly choose the same number of normal images in normal path
    save them into outlier path
    '''
    train_img_paths = glob('../data/train/'+"*_sat.jpg")
    ig = image_gen1(train_img_paths)
    for i in range(len(train_img_paths) - 1):
        if i % 500 == 0:
           print(i)
        img, mask, img_id = next(ig)
        # if the image is outlier
        if len(np.where(mask[:,:,0].T.flatten()!=0)[0] ) == 0:
            if os.path.exists(outlier_path):
              imsave(os.path.join(outlier_path, img_id + '_outlier_sat.jpg'), img)
              #imsave(os.path.join(outlier_path, img_id + '_msk.png'), mask)
              
              # augment outlier
              # horizontal flip
              imsave(os.path.join(outlier_path, img_id + '_a_outlier_sat.jpg'), img[:,::-1])
              #imsave(os.path.join(outlier_path, img_id + 'a_msk.png'), mask)
              # vertical flip
              imsave(os.path.join(outlier_path, img_id + '_b_outlier_sat.jpg'), img[::-1,:])
              #imsave(os.path.join(outlier_path, img_id + 'b_msk.png'), mask) 
              
            else:
              print("path not exist...")
        else:
            if os.path.exists(normal_path):
              imsave(os.path.join(normal_path, img_id + '_sat.jpg'), img)
              # imsave(os.path.join(normal_path, img_id + '_msk.png'), mask)
            else:
              print("path not exist...")
     
    normal_paths = glob(normal_path+"/*_sat.jpg")
    outlier_paths = glob(outlier_path+"/*_sat.jpg")
    normal_imgs = np.random.choice(normal_paths,len(outlier_paths)-1)
    
    ig1 = image_gen1(normal_imgs)
    for i in range(len(normal_imgs)-1):
      img, mask, img_id = next(ig1)
      imsave(os.path.join(outlier_path, img_id + '_sat.jpg'), img)
      #imsave(os.path.join(outlier_path, img_id + '_msk.png'), mask) 
    

      

In [0]:
os.chdir("/content/Satellite-Segmentation/data/")
! rm -rf outlier
! rm -rf normal
!mkdir outlier
!mkdir normal
os.chdir("/content/Satellite-Segmentation/src/")
aug_outlier('/content/Satellite-Segmentation/data/outlier', '/content/Satellite-Segmentation/data/normal')

In [0]:
outlier_img_paths = glob('/content/Satellite-Segmentation/data/outlier'+"/*_sat.jpg")
print(len(outlier_img_paths))
normal_paths = glob('/content/Satellite-Segmentation/data/normal'+"/*_sat.jpg")
print(len(normal_paths))

9648
9058


In [0]:
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

Using TensorFlow backend.


In [0]:
from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

def get_img_id(img_path):
  img_basename = os.path.basename(img_path)
  img_id = os.path.splitext(img_basename)[0][:-len('_sat')]
  return img_id

def img_gen(img_paths, img_size=IMG_SIZE):
  #Iterate over all image paths
  for i, img_path in enumerate(img_paths):
    img_id = get_img_id(img_path)
    img = imread(img_path)/255
    if 'out' in os.path.basename(img_path):
      label = [0,1]
    else:
      label = [1,0]
    
    img = resize(img, img_size, preserve_range = True)
#     mask = resize(mask, img_size, mode='constant', preserve_range = True)
   
    yield img, label

In [0]:
import numpy as np

# Keras takes its input in batches 
# (i.e. a batch size of 32 would correspond to 32 images and 32 masks from the generator)
# The generator should run forever
def image_batch_generator(img_paths, batchsize=32):
    while True:
        ig = img_gen(img_paths)
        batch_img, batch_label = [], []
        
        for img, label in ig:
            # Add the image and mask to the batch
            batch_img.append(img)
            batch_label.append(label)
            # If we've reached our batchsize, yield the batch and reset
            if len(batch_img) == batchsize:
                yield np.stack(batch_img, axis=0), np.stack(batch_label, axis=0)
                batch_img, batch_label = [], []
        
        # If we have an nonempty batch left, yield it out and reset
        if len(batch_img) != 0:
            yield np.stack(batch_img, axis=0), np.stack(batch_label, axis=0)
            batch_img, batch_label = [], []



In [0]:
from sklearn.model_selection import train_test_split
import random

BATCHSIZE = 8
outlier_img_paths = glob(os.path.join('./647outlier','*.jpg'))
train_img_paths = outlier_img_paths

random.shuffle(train_img_paths)


# Split the data into a train and validation set
train_img_paths, val_img_paths = train_test_split(train_img_paths, test_size=0.15)
print(train_img_paths)
print(len(train_img_paths))
print(len(val_img_paths))

# Create the train and validation generators
traingen = image_batch_generator(train_img_paths, batchsize=BATCHSIZE)
valgen = image_batch_generator(val_img_paths, batchsize=BATCHSIZE)

print(next(traingen))

def calc_steps(data_len, batchsize):
    return (data_len + batchsize - 1) // batchsize

# Calculate the steps per epoch
train_steps = calc_steps(len(train_img_paths), BATCHSIZE)
val_steps = calc_steps(len(val_img_paths), BATCHSIZE)
print("train steps {}".format(train_steps))
print("val steps {}".format(val_steps))

['./647outlier/37555_sat.jpg', './647outlier/3556a_out.jpg', './647outlier/10027_sat.jpg', './647outlier/39337_out.jpg', './647outlier/44638_sat.jpg', './647outlier/13267b_out.jpg', './647outlier/44839_out.jpg', './647outlier/15344_sat.jpg', './647outlier/7581b_out.jpg', './647outlier/50484b_out.jpg', './647outlier/31921_sat.jpg', './647outlier/18107_sat.jpg', './647outlier/12248_sat.jpg', './647outlier/38907b_out.jpg', './647outlier/52379b_out.jpg', './647outlier/26757a_out.jpg', './647outlier/34965_sat.jpg', './647outlier/11388_out.jpg', './647outlier/44751b_out.jpg', './647outlier/30214_out.jpg', './647outlier/13351b_out.jpg', './647outlier/52380a_out.jpg', './647outlier/5477a_out.jpg', './647outlier/33705_sat.jpg', './647outlier/23944a_out.jpg', './647outlier/39082_sat.jpg', './647outlier/27860_sat.jpg', './647outlier/12742a_out.jpg', './647outlier/8129b_out.jpg', './647outlier/17583a_out.jpg', './647outlier/2975a_out.jpg', './647outlier/37763a_out.jpg', './647outlier/15224a_out.jp

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.applications.densenet import DenseNet121
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


IMAGE_SIZE    = IMG_SIZE
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training



net = DenseNet121(include_top=False, weights=None, input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
    
model = Model(inputs=net.input, outputs=output_layer)

# for layer in model.layers[:FREEZE_LAYERS]:
#     layer.trainable = False
# for layer in model.layers[FREEZE_LAYERS:]:
#     layer.trainable = True
    
model.compile(Adam(5e-5), loss="binary_crossentropy", metrics=["binary_accuracy"])
model.summary()

history = model.fit_generator(
    traingen, 
    steps_per_epoch=train_steps, 
    epochs=30, # Change this to a larger number to train for longer
    validation_data=valgen, 
    validation_steps=val_steps, 
    verbose=1,
    max_queue_size=5  # Change this number based on memory restrictions
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_12 (ZeroPadding2 (None, 230, 230, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_12[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


Epoch 1/30
394/394 [==============================] - 201s 510ms/step - loss: 0.9613 - binary_accuracy: 0.5522 - val_loss: 0.7487 - val_binary_accuracy: 0.4919
Epoch 2/30
394/394 [==============================] - 181s 459ms/step - loss: 0.8368 - binary_accuracy: 0.6077 - val_loss: 0.8706 - val_binary_accuracy: 0.6014
Epoch 3/30
394/394 [==============================] - 179s 455ms/step - loss: 0.7216 - binary_accuracy: 0.6652 - val_loss: 0.8937 - val_binary_accuracy: 0.6230
Epoch 4/30
394/394 [==============================] - 181s 460ms/step - loss: 0.6227 - binary_accuracy: 0.6934 - val_loss: 0.9476 - val_binary_accuracy: 0.5655
Epoch 5/30
394/394 [==============================] - 182s 461ms/step - loss: 0.5472 - binary_accuracy: 0.7490 - val_loss: 1.6555 - val_binary_accuracy: 0.5368
Epoch 6/30
394/394 [==============================] - 182s 461ms/step - loss: 0.4476 - binary_accuracy: 0.7880 - val_loss: 1.5188 - val_binary_accuracy: 0.5440
Epoch 7/30
394/394 [====================

In [0]:
model.save('od-7-0.9786-0.6913-647aug.h5')

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [0]:

# outlier detector
# https://drive.google.com/open?id=1yVLwUH94qzpTQK7aoPPeyvtUx9LitjFD
# od-resnet50-binary-10-0.7456-0.8398-aug-id1.h5

# outlier
# https://drive.google.com/open?id=12tuX2zD_Dq6svJemDxOHr7VX21RSTe_3
# od-7-0.9786-0.6913-647aug.h5
outlier_model = drive.CreateFile({'id':'12tuX2zD_Dq6svJemDxOHr7VX21RSTe_3'})
outlier_model.GetContentFile('od-7-0.9786-0.6913-647aug.h5')


In [0]:

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.models import load_model



resnet_model = load_model('od-7-0.9786-0.6913-647aug.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [0]:
# Get and resize test images
path_to_val = '../data/val'
glob_val_imgs = os.path.join(path_to_val, '*_sat.jpg')
val_img_paths = glob(glob_val_imgs)
test_id = [get_img_id(val_img_paths[i]) for i in range(len(val_img_paths))]

def val_img_gen(val_img_paths,img_size=IMG_SIZE):
  for i, val_img_path in enumerate(val_img_paths):
    img = imread(val_img_path)/255
    img = resize(img, img_size, preserve_range = True)
    img = np.reshape(img, (1,224,224,3))
    yield img
    

# Predict on test
print(val_img_paths)
vig = val_img_gen(val_img_paths)


result = resnet_model.predict_generator(vig,steps = len(val_img_paths), max_queue_size=5)


['../data/val/76357_sat.jpg', '../data/val/77807_sat.jpg', '../data/val/78969_sat.jpg', '../data/val/72392_sat.jpg', '../data/val/79440_sat.jpg', '../data/val/73331_sat.jpg', '../data/val/72980_sat.jpg', '../data/val/70170_sat.jpg', '../data/val/77339_sat.jpg', '../data/val/77838_sat.jpg', '../data/val/71284_sat.jpg', '../data/val/72510_sat.jpg', '../data/val/71520_sat.jpg', '../data/val/72761_sat.jpg', '../data/val/73771_sat.jpg', '../data/val/72670_sat.jpg', '../data/val/73131_sat.jpg', '../data/val/77485_sat.jpg', '../data/val/76907_sat.jpg', '../data/val/75392_sat.jpg', '../data/val/73470_sat.jpg', '../data/val/70368_sat.jpg', '../data/val/75234_sat.jpg', '../data/val/73315_sat.jpg', '../data/val/68919_sat.jpg', '../data/val/78674_sat.jpg', '../data/val/76496_sat.jpg', '../data/val/74584_sat.jpg', '../data/val/73095_sat.jpg', '../data/val/76541_sat.jpg', '../data/val/75027_sat.jpg', '../data/val/79160_sat.jpg', '../data/val/77885_sat.jpg', '../data/val/71960_sat.jpg', '../data/val/

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [0]:
print(result.shape)

(2169, 2)


In [0]:
import pandas as pd
# Create submission DataFrame
def create_submission(csv_name, predictions, image_ids):
    """
    csv_name -> string for csv ("XXXXXXX.csv")
    predictions -> numpyarray of size (num_examples, height, width)
                In this case (num_examples, 512, 512)
    image_ids -> numpyarray or list of size (num_examples,)
    
    predictions[i] should be the prediction of road for image_id[i]
    """
    sub = pd.DataFrame()
    sub['ImageId'] = image_ids
    encodings = []
    num_images = len(image_ids)
    count = 0
    for i in range(num_images):
      if predictions[i][1] >= 0.999:
        count += 1
        encodings.append(1)
      else:
        encodings.append(0)
        
    sub['is_outlier'] = encodings
    #sub['Height'] = [512]*num_images Nonger needed for DICE Scoring
    #sub['Width'] = [512]*num_images Nonger needed for DICE Scoring
    sub.to_csv(csv_name, index=False)
    print(count)


create_submission("od-7-0.9786-0.6913-647aug-id0999-submission.csv", result, test_id)

48


In [0]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

In [0]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)
  
# Build U-Net model
inputs = Input((512, 512, 3))
s = inputs

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

x = Flatten()(c5 )
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax', name='softmax')(x)

import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

model = Model(inputs=[inputs], outputs=[output_layer])
model.compile(Adam(1e-4), loss=binary_crossentropy, metrics=["binary_accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512, 512, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 512, 512, 16)      448       
_________________________________________________________________
dropout_7 (Dropout)          (None, 512, 512, 16)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 512, 512, 16)      2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 256, 256, 32)      4640      
_________________________________________________________________
dropout_8 (Dropout)          (None, 256, 256, 32)      0         
__________

In [0]:
print(next(traingen))

In [0]:
clr = CyclicLR(base_lr=BASE_LR, max_lr=MAX_LR,
                                step_size=STEP_SIZE, mode=MODE)

checkpointer = ModelCheckpoint(MODEL_NAME, verbose=1)


# Train the model
history = model.fit_generator(
    traingen, 
    steps_per_epoch=train_steps, 
    epochs=EPOCH_NUM, # Change this to a larger number to train for longer
    validation_data=valgen, 
    validation_steps=val_steps, 
    verbose=1,
    max_queue_size=5,  # Change this number based on memory restrictions
    callbacks = [checkpointer, clr]
)


Output hidden; open in https://colab.research.google.com to view.